In [1]:
import os
import numpy as np
import xarray as xr
import pickle
import healpy as hp
import easygems.healpix as egh

from functions import change_zoom_level

figpath = 'figures/dry_days'

In [2]:
bounds_all = np.arange(0, 1000.01, .01)
bounds_all = np.concatenate([bounds_all, [9999]])  # ensure the last bin covers all the rest

## IFS

In [3]:
basepath = '/work/uc1275/LukasBrunner/data/SubGridVariability'

fn = 'IFS-9-FESOM-5-production_pr_z9_day_2021-2049.nc'
da = xr.open_dataset(os.path.join(basepath, 'input', fn))['pr'].load()

da *= 1000  # original units: m (implicit: per day)
da.attrs['units'] = 'mm/day'

In [4]:
da_z = {9: da}

for zoom in range(8, -1, -1):
    da_z[zoom] = change_zoom_level(da_z[zoom+1], gridn='value', z_out=zoom)

In [ ]:
for zoom in da_z:
    print('{}: {:.2f}'.format(zoom, da_z[zoom].values.max().item()))

In [21]:
for zoom in da_z:
    print('{}: {:.1e}'.format(zoom, da_z[zoom].values.ravel().size))

9: 3.3e+10
8: 8.3e+09
7: 2.1e+09
6: 5.2e+08
5: 1.3e+08
4: 3.3e+07
3: 8.1e+06
2: 2.0e+06
1: 5.1e+05
0: 1.3e+05


9: 3.332e+10
8: 8.330e+09
7: 2.082e+09
6: 5.206e+08
5: 1.302e+08
4: 3.254e+07
3: 8.135e+06
2: 2.034e+06
1: 5.084e+05
0: 1.271e+05

In [6]:
# get data on all zoom levels
da_z = {9: da}

for zoom in range(8, -1, -1):
    da_z[zoom] = change_zoom_level(da_z[zoom+1], gridn='value', z_out=zoom)

# calculate histogram on all zoom levels
# NOTE: heavy lifting, runs for a couple of hours
hist_all_z = {zoom: np.histogram(da_z[zoom].values.ravel(), bins=bounds_all)[0] / da_z[zoom].values.ravel().size for zoom in range(0, 10, 1)}

with open('/work/uc1275/LukasBrunner/data/SubGridVariability/results/pr_binned-frequencies_ifs_2021-2049.pkl', 'wb') as ff:
    pickle.dump(hist_all_z, ff)

In [23]:
del da_z

## ICON

In [ ]:
basepath = '/work/uc1275/LukasBrunner/data/SubGridVariability'

fn = 'ICON-ngc4008_pr_z9_day_2020-2049.nc'
da = xr.open_dataset(os.path.join(basepath, 'input', fn))['pr'].load()
da = da.sel(time=slice('2021', None))

da *= 60*60*24
da.attrs['units'] = 'mm/day'  # from mm/s

In [ ]:
da_z = {9: da}

for zoom in range(8, -1, -1):
    da_z[zoom] = change_zoom_level(da_z[zoom+1], gridn='value', z_out=zoom)

In [ ]:
for zoom in da_z:
    print('{}: {:.3e}'.format(zoom, da_z[zoom].values.ravel().size))

In [ ]:
# get data on all zoom levels
da_z = {9: da}

for zoom in range(8, -1, -1):
    da_z[zoom] = change_zoom_level(da_z[zoom+1], gridn='x', z_out=zoom)

# calculate histogram on all zoom levels
# NOTE: heavy lifting, runs for a couple of hours
hist_all_z = {zoom: np.histogram(da_z[zoom].values.ravel(), bins=bounds_all)[0] / da_z[zoom].values.ravel().size for zoom in range(0, 10, 1)}

with open('/work/uc1275/LukasBrunner/data/SubGridVariability/results/pr_binned-frequencies_icon_2021-2049.pkl', 'wb') as ff:
    pickle.dump(hist_all_z, ff)